# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint

#### To be done in the Lab

The objective of this experiment is to perform sentimental analysis.

In this experiment we will be using twitter dataset as training data and crawled realtime tweets for testing. 

The Ground truth is 1 for positive tweet and 0 for negative tweet.

Few examples of positive and negative tweets are:

**Few Positive Tweets: **
1.  @Msdebramaye I heard about that contest! Congrats girl!!
2. UNC!!! NCAA Champs!! Franklin St.: I WAS THERE!! WILD AND CRAZY!!!!!! Nothing like it...EVER http://tinyurl.com/49955t3

**Few Negative Tweets:**
1. no more taking Irish car bombs with strange Australian women who can drink like rockstars...my head hurts.
2. Just had some bloodwork done. My arm hurts

### Data Source

https://www.kaggle.com/c/twitter-sentiment-analysis2/data


## Exercise 1: (2 marks)

The first exercise is cleaning the tweets.
Perform preprocessing as required.

Complete the functon : preprocess_tweets 

Input or Arguement to the function : tweet as a string 

Return value: processed tweet as string 

Hint: Use regular expressions
* convert the all the cases into lower case
  + look at lower()
* Replace any urls with the word "URL"
  + Hint : 
      - re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',"Tweet") (re is python regular expression package)
* convert the username to "AT_USER", consider any word that starts with @ as user name
  + Hint : 
      - re.sub('@[^\s]+','AT_USER',"Tweet")
* Remove multiple whitespaces with a single white space
  + Hint :
      - re.sub('[\s]+', ' ', tweet)
* Replace hashtag words (#word) with just the words (word)
  + Hint : 
      - re.sub(r'#([^\s]+)', r'\1', "tweet")
      
* TEST CASE :
    + given the tweet "@V_DEL_ROSSI: Me         #dragging myself to the gym https://t.co/cOjM0mBVeY"
    + output should be "AT_USER me dragging myself to the gym URL"

In [1]:
import re
import gensim
# Operating System
import os
# Regular Expression
import re
# nltk packages
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
# Basic Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# PCA Package
from sklearn.decomposition import PCA

In [2]:
def preprocess_tweets(tweet):
    tweet=tweet.lower()
    tweet = re.sub('((www.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet=re.sub('@[^\s]+','AT_USER',tweet)
    tweet=re.sub('[\s]+', ' ', tweet)
    tweet=re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet=re.sub('&lt;','',tweet)#xml tag
    tweet=re.sub('&amp;','',tweet)
    tweet=re.sub(':o','smile',tweet) #replace smileys with good or bad emotion
    tweet=re.sub(':-\|','speechless',tweet)
    tweet=re.sub('[.!,-]+', '', tweet)
    return tweet

In [3]:
import pandas as pd
data = pd.read_csv('twitter_train.csv',encoding='latin-1',converters={'SentimentText':preprocess_tweets})
pw=pd.read_csv('positive-words.txt').values
nw=pd.read_csv('negative-words.txt').values


In [4]:
pw[3][0]

'abundant'

In [5]:
for i in range(100):
    print(data.values[i])

[1 0 ' is so sad for my apl friend']
[2 0 ' i missed the new moon trailer']
[3 1 ' omg its already 7:30 smile']
[4 0
 "  omgaga im sooo im gunna cry i've been at this dentist since 11 i was suposed 2 just get a crown put on (30mins)"]
[5 0 ' i think mi bf is cheating on me t_t']
[6 0 ' or i just worry too much? ']
[7 1 ' juuuuuuuuuuuuuuuuussssst chillin']
[8 0 ' sunny again work tomorrow speechless tv tonight']
[9 1 ' handed in my uniform today  i miss you already']
[10 1 ' hmmmm i wonder how she my number AT_USER']
[11 0 ' i must think about positive']
[12 1 ' thanks to all the haters up in my face all day 112102']
[13 0 ' this weekend has sucked so far']
[14 0 ' jb isnt showing in australia any more']
[15 0 ' ok thats it you win']
[16 0 '  this is the way i feel right now']
[17 0
 " awhhe man i'm completely useless rt now funny all i can do is twitter URL"]
[18 1
 " feeling strangely fine now i'm gonna go listen to some semisonic to celebrate"]
[19 0 ' huge roll of thunder just nowso

## Exercise 2: (3 marks)

Tokenize the processed tweets to make a tweet into a list of words and make sure that no punctuations are returned. so that it can be used in the next steps to represent the tweet as a feature vector. Remove the Stops words, if necessary

Complete the functon : word_tokenizer 

Input or Arguement to the function : processed tweet

Return value: list of words without any punctuations

TEST CASE :

Given an input :
    "Neither Man, nor machine can replace its creator. really?."
    
Result : 
    ['neither', 'man', 'nor', 'machine', 'replace', 'creator', 'hahaha']

In [6]:
stopWords = pd.read_csv('stopwords.txt').values
stemmer = SnowballStemmer("english")
vocabulary = set([])

In [7]:
def word_tokenizer(tweet):
    words = re.findall(r'(\b[A-Za-z][a-z]{2,15}\b)', tweet)
    words = [ stemmer.stem(word.lower()) for word in words if not word.lower() in stopWords]
    return words

In [8]:
data.values.shape[0]

99989

In [9]:
for i in range(100):
    words= word_tokenizer(data.values[i][2])
    for word in words:
          vocabulary.add(word)
vocabularyList=list(vocabulary)

In [10]:
# add postive words count, negative wourd count, neutral, and tweet (label)
voclength=len(vocabulary)
fvlength=voclength+4

## Exercise 3: (5 marks)

Using the list of words from the above the step, 
* represent the tweet as a feature vector using bag of words/Word2vec

Hint : counts of postive/negative/neutral words as three features can also be used

In [11]:
def getfeaturevector(tokenized_tweet,sentiment):
    feature_vector=np.zeros(fvlength)
    for word in tokenized_tweet: 
        flag=0
        for i in range(len(pw)):
            if word == pw[i][0]:
                feature_vector[voclength]+=1
                flag=1
        for i in range(len(nw)):
            if word == nw[i][0]:
                feature_vector[voclength+1]+=1
                flag=-1
        if flag==0:
            feature_vector[voclength+2]+=1
        feature_vector[voclength+3]=sentiment
    return feature_vector

In [12]:
#fv = getfeaturevector(word_tokenizer(data.values[20][2]))
#fv[2133],fv[908],fv[len(vocabulary)]

In [13]:
#for i in range(data.values):
#    fv = getfeaturevector(word_tokenizer(data.values[20][2]))

In [14]:
#len(pw)

In [15]:
#data.values[20][2]

In [16]:
finaldata=[]
for i in range(100):
    finaldata.append(getfeaturevector(word_tokenizer(data.values[i][2]),data.values[i][1]))

## Exercise 4: (Marks : 5 ) 


Load the given training data and use the above functions you created to process, to tokenise and to get feature vector.

Considering the feature vector as input to the classifier, Train a classifier to classify the sentiment of the tweet correctly.

Divide the training data into two sets, to validate your classifier

In [17]:
print
# Your Code Hereimport random
train = []
test = []
import random
def randomize():
    TRAIN_TEST_RATIO = 0.8
    data = finaldata
    #print(data)
    for one in data:
        #print(one)
        if random.random() < TRAIN_TEST_RATIO:
            train.append(one)
        else:
            test.append(one)
    print (test[:10])

    

In [18]:
randomize()

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0

In [19]:
import math
import collections
def dist(a, b):
    sqSum = 0
    for i in np.arange( len(a)):
        sqSum += (a[i] - b[i]) ** 2
    return math.sqrt(sqSum)
# ------------------------------------------------ #
# We are assuming that the label is the last field #
# If not, munge the data to make it so!            #
# ------------------------------------------------ #
def kNN(k, train, given):
    distances = []
    for t in train:
        distance=(dist(t[:-1], given), str(int(t[-1])))
        #print(distance)
        distances.append(distance)
    distances.sort()
    
    return distances[:k]

def kNN_classify(k, train, given):
    tally = collections.Counter()
    #print(tally)
    for nn in kNN(k, train, given):
        tally.update(nn[-1])
    return tally.most_common(1)[0]

In [20]:
test

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [21]:
def KNN_accuracy(Kvalue):
    num = 0
    print("calculating accuracy")
    for t in test:
        prediction = kNN_classify(Kvalue, train, t)[0]
        #print("prediction, label is", prediction, t[134])
        if (int(prediction) == int(t[-1])):
            num += 1
        #print(kNN_classify(5, train, t)[0], t[-1])
    #print(num/len(test))
    return (num/len(test))

In [22]:
KNN_accuracy(3)

calculating accuracy


0.7894736842105263

## Exercise 5: (Marks : 5)

#### Twitter crawling using tweepy

Use tweepy to get the tweets on real time, which is used as test data for the classifier.

## Requirements: 

Twitter account

Create a twitter account if you don't have one by going to the link given below:

https://twitter.com/i/flow/signup

## Installation

Tweepy: tweepy is the python client for the official Twitter API.
Install it using following pip command:

In [23]:
!pip install tweepy

The tweets need to be gathered so as to perform Sentiment analysis on those tweets. They can be fetched from Twitter using the Twitter API. 

In order to fetch tweets through Twitter API, one needs to register an App through their twitter account. Follow these steps for the same:
<ul>
<li>Open the link given below to create a App through the twitter account.
    https://apps.twitter.com
<li>click the button: ‘Create New App’
<li>Fill the application details. You can leave the callback url field empty.
<li>Once the app is created, you will be redirected to the app page.
<li>Open the ‘Keys and Access Tokens’ tab.
<li>Copy ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’.
</ul>

In [24]:
#Replace with your ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’ below. 
consumer_key = "sjqCktpagvR4NjTsUR3DcA7pp"
consumer_secret = "dqxiWPhqt7n7uR0tGqA18bPCRsXxtEPVNg3HmOtfmzjjjMEXwa"
access_token =  "968060979681681408-kTK6GHKeBRr3oyEalq3NwQKtC9Vn4fS"
access_secret = "pICNG5EIzlBwf0vmSLCitRmtloKr1J1Hmd6w8JixWDmKU"

Run the code below to authenticate your application.

In [25]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)    

## Tweepy Cursor

The below code gives the search results from twitter for the search string passed to the keyword arguement "q" in the tweepy.Cursor. The number passed to the items method of tweepy.Cursor indicates that it gives 100 such tweets, if available

In [26]:
consumer_key = 'BhmQiexbkLED5X6MgfPbeG6RC'
consumer_secret = '9CfC1dNScx4yESOe2dKEFG1QoTTJ3sHLI5lg7ZYTM8HFbx3a8p'
access_token = '1028201742-my5TYzCYTxhLSYiUdJ5DK7EeDV0ajK4bic7izqb'
access_secret = 'SHr4EVGWYZqVEfijVIJ1M6fFWfoOWkJ4P1laOwqGLrHdV'


Also apply the preprocessing steps and obtain the feature vectors for the crawled twitter data.
Classify the crawled tweets by passing its feature vector to the trained classifier.

In [27]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)    


import tweepy
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        tweet = status.text
        t = getfeaturevector(word_tokenizer(tweet),0)
        #print(t[-1],t[-2],t[-3],t[-4])
        prediction = int(kNN_classify(3, finaldata, t)[0])
        if 0 >= prediction :
            print('POSITIVE',prediction,status.text)
        else:
            print('NEGATIVE',prediction,status.text)
            

In [28]:


myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)


In [29]:
myStream.filter(track=['dhoni'])

POSITIVE 0 RT @DippakMehrotra: Mahendra Singh Dhoni Who's a Inspiration to One &amp; All will Appear in STAR REIMAGINE's Exclusive On Star Sports India at…
POSITIVE 0 RT @imPakistaniLAD: Happy Birthday MS Dhoni ♥ https://t.co/t31psKiziy
POSITIVE 0 happy birthday  dhoni https://t.co/BXHf4pT4CV
POSITIVE 0 RT @CSKFansOfficial: Tribute to Thala M.S.Dhoni from the #WhistlePoduArmy.
Calling all the Superfans to send in their entries using #ThalaS…
POSITIVE 0 Today Is Best Day Bcoz..Today B'day of God Of Cricket 
Sir ...( *M.S Dhoni* )
Wish U Happy Birthday Day 
Sir.. ( *M.S.Dhoni* ) 
@msdhoni
POSITIVE 0 Mahendra singh dhoni is super cool captain and he is coming to the  STAR REIMAGINE show
 #MSDReimagined @starsportsindia
POSITIVE 0 RT @Gobinath_C: Happy Birthday The Captain...The Leader... The King... one and only #MSDhoni #MSD #Dhoni #HappyBirthdayMSD #HappyBirthdayDh…
POSITIVE 0 RT @BCCI: SPECIAL: From his teammates to someone very very special, wishes galore for @msdhoni as he celebrates

POSITIVE 0 Ziva😍😂 https://t.co/4nShpWcijN
POSITIVE 0 RT @shrutisaxena95: 🎈Happy birthday 🎂to my🎈 favourite cricketer🎂 M.S. DHONI👌 🎂🎁🎁🎁💖💖💖💖
@MSDhoni7__ https://t.co/7IPihK75e7
POSITIVE 0 RT @prakashachary1: Wishing You A Many Many Happy Returns Of The Day🎂 #CaptainCool @msdhoni Sir Have A Wonderful Year Going Ahead,I Wish Yo…
POSITIVE 0 RT @Cricketracker: MS Dhoni - A complete package. #HappyBirthdayMSDhoni https://t.co/Ohk0O6r3Pf
POSITIVE 0 @dashhtweets Does that mean dhoni is not a cricketer?
POSITIVE 0 M s Dhoni cricket ka jadugar
Happy birthday today
POSITIVE 0 RT @Saurabh10713687: #MSDReimagined Happy birthday Dhoni sir https://t.co/Cd7LKBe1iP
POSITIVE 0 Watch Dhoni candidly describing himself as a person - on and off the field at STAR REIMAGINE on @StarSportsIndia to… https://t.co/GjiI41o1QF
POSITIVE 0 Happy Birthday THALA DHONI :D @msdhoni 

https://t.co/iozf99rwox

#HBDTHALADHONI #CSK #dhonibirthday #Dhoni https://t.co/zJQDk4Piy7
POSITIVE 0 @ImRaina @msdhoni Happy birthday  MS D

POSITIVE 0 RT @BrokenCricket: Smart captaincy...
Swift stumping..
Last ball six..
Late moment finish...
Standing aside..
Helicopter shot..
Fast runnin…
POSITIVE 0 @sud_tyagi Dhoni Hater !😏😆
POSITIVE 0 AALAPORAAN TAMIZHAN - MS DHONI VERSION - TRIBUTE TO DHONI | Captain Dhoni Official - Saran Supermass: https://t.co/Dqlp6R3vJD via
POSITIVE 0 RT @BrokenCricket: Dhoni after 500 matches:

Runs: 16330
100s: 16
  50s: 102
Sixes: 342
Catches: 602
Stumpins: 178
Captain: 331
Wins: 178
POSITIVE 0 Dhoni finishes of in style it’s a magnificent strike into the crowd India lifts the World Cup after 28 years party… https://t.co/b5gz0xlDW8
POSITIVE 0 RT @Cricketracker: One of the most successful captains of all time, MS Dhoni turns 37 today. #HappyBirthdayMSDhoni https://t.co/EDdOPncMEQ
POSITIVE 0 My favourite Captain super cool M S Dhoni  is coming to the  STAR REIMAGINE show 
 #MSDReimagined so don't miss to… https://t.co/3joU0WZkRv
POSITIVE 0 @imVkohli @AnushkaSharma @msdhoni Happy birthday M.s Dhon

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.